# Learning the Handwritten Digits

You're going to build a model on the digits dataset, a sample dataset that comes pre-loaded with scikit learn. The digits dataset consist of 8x8 pixel handwritten digits from 0 to 9:

<img src='data/digs.png' width="600" height="300" align="center"/>

You want to distinguish between each of the 10 possible digits given an image, so we are dealing with multi-class classification.

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
X = np.load('Digits/digits_pixels.npy')
y = np.load('Digits/digits_target.npy')

In [6]:
y = to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)